**SKLEARN DECISION TREE REGRESSOR working with 1985 to 1990 data.**

In [1]:
# most basic imports that I would need, need more to visualize
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
import category_encoders as ce
import xgboost as xgb




Next, must load the data!
t1 represents the data we are working on predicting, while t2 represents the expected values of the prediction. t2 will be used primarily
for calculating the loss function. Switch to 85 and 90

In [2]:

# load all data
raw_data_t1 = pd.read_csv("/Users/dylanvanbramer/indresearch/xu/deep_learning/bci_census/bci_1985.csv")
raw_data_t2 = pd.read_csv("/Users/dylanvanbramer/indresearch/xu/deep_learning/bci_census/bci_1990.csv")

# load the first quadrat that we will be working with
quad1_t1 = raw_data_t1.loc[raw_data_t1['quadrat']==1]
quad1_t2 = raw_data_t2.loc[raw_data_t2['quadrat']==1]

# we only care about the tree IDs and DBHs of quad1_t2
expected_labels = quad1_t2[['treeID', 'dbh']]
expected_labels = expected_labels.rename(columns={"dbh": "dbh2", "treeID": "treeID2"})

#quad1_t1.head()

/var/folders/zf/kbwgwbdd4gjfhdxc9cnw8qgm0000gn/T/ipykernel_43399/3260900854.py:2: DtypeWarning: Columns (14,16) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_data_t1 = pd.read_csv("/Users/dylanvanbramer/indresearch/xu/deep_learning/bci_census/bci_1985.csv")
/var/folders/zf/kbwgwbdd4gjfhdxc9cnw8qgm0000gn/T/ipykernel_43399/3260900854.py:3: DtypeWarning: Columns (14,16) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_data_t2 = pd.read_csv("/Users/dylanvanbramer/indresearch/xu/deep_learning/bci_census/bci_1990.csv")


Simplify data: extract important features. Next step: remove trees that are dead in either census, other cleaning this way to get rid of data errors. This could involve no species listed, or zero or NaN DBH. Later on, will need the actual date too.

In [3]:
# simplify the data to have less features
quad1_t1 = quad1_t1[['treeID', 'sp', 'gx', 'gy', 'dbh']]
quad1_t1 = quad1_t1.rename(columns={"dbh": "dbh1", "treeID":"treeID1"})

Create a list of all of the species present and index them in a list. 

NOTE: Now am exploring different ways to encode the species so that they are  ordered nominally!

Probably makes sense to use hash encoding because there are so many possible options for species (just in this one quadrat, 85 species are present)

In [4]:
encoder= ce.BinaryEncoder(cols=['sp'],return_df=True)
data_encoded=encoder.fit_transform(quad1_t1)

data_encoded.head()

,treeID1,sp_0,sp_1,sp_2,sp_3,sp_4,sp_5,sp_6,gx,gy,dbh1
8223,8224,0,0,0,0,0,0,1,1.6,24.700001,294.0
230867,230868,0,0,0,0,0,1,0,2.8,20.400000,NaN
230971,230972,0,0,0,0,0,1,1,0.3,24.100000,15.0
230972,230973,0,0,0,0,1,0,0,0.2,24.600000,25.0
230973,230974,0,0,0,0,1,0,1,0.8,24.700001,15.0






species_list = []
#df.loc[df['A'] > 2, 'B'] 
#print (range(len(quad1_t1)))
for i in quad1_t1.index:
    current = quad1_t1['sp'][i] 
    #print (current)
    if current not in species_list:
            species_list.append(current)
    quad1_t1['sp'][i]  = species_list.index(current)

print (len(species_list))
#quad1_t1.head(30)
#print(species_list)
#quad1_t1.head()

In [6]:
#quad1_t1.head(30)
#print(species_list)

NEED TO FIX SO THAT I DONT GET THE ERRORS ABOVE! Tried to use df.loc[] but ran into many errors. Also note above that the indices are one less than the tree ID - why is that??

https://stackoverflow.com/questions/20625582/how-to-deal-with-settingwithcopywarning-in-pandas

In [7]:
df_combined = pd.concat([quad1_t1, expected_labels], axis=1)

# Drop rows with any NaN values
df_combined_clean = df_combined.dropna()

# Separate the cleaned DataFrame and labels
df_clean = df_combined_clean[quad1_t1.columns]
labels_clean = df_combined_clean[expected_labels.columns]
#df_combined_clean.head()
#df_clean.head()
#labels_clean.head()




Split data into training and validation sets. Here, there's a 70/30 split. Can use scikit-learn's algorithm. 

In [8]:
# convert both from above (no null) to numpy arrays
quad1_t1 = df_clean.to_numpy()
expected_labels = labels_clean.to_numpy()


In [12]:
quad1_t1 = quad1_t1.astype(np.float32)
expected_labels = expected_labels.astype(np.float32)


In [13]:
n, d = quad1_t1.shape
#cutoff = int(np.ceil(0.7 * n))
# indices of training samples
#xTr = quad1_t1[:cutoff,:]
#yTr = expected_labels[:cutoff]
# indices of testing samples
#xTv = quad1_t1[cutoff:,:]
#yTv = expected_labels[cutoff:]

# above was a way to compute it by hand, but instead can use sklearn
X_train, X_test, y_train, y_test = train_test_split(quad1_t1, expected_labels, test_size=0.3)
print (X_train.dtype)

float32


NEXT STEPS:
1) find the distance between a focal tree and each of its neighbors (sort).
2) pick the 50 closest neighbors.
3) create a matrix with 150 dimensions:
    FOR EACH OF 50 TREES:
    distance to the neighbor
    species of neighbor
    dbh of neighbor

    

In [14]:
# first convert to numpy arrays
#X_train = X_train.to_numpy()
#X_test = X_test.to_numpy()
#y_train = y_train.to_numpy()
#y_test = y_test.to_numpy()

#print(type(X_train))

# then convert those numpy arrays to tensors
X_train = torch.from_numpy(X_train)
X_test = torch.from_numpy(X_test)
y_train = torch.from_numpy(y_train)
y_test = torch.from_numpy(y_test)


#having some errors here, but will be fairly easy to debug I think

<class 'numpy.ndarray'>


try using sklearn decision tree regressor on just this simple data!

In [19]:
tree = DecisionTreeRegressor()
tree.fit(X_train[:,1:], y_train[:,1])

DecisionTreeRegressor()

In [22]:
identifiers = X_test[:, 0]  
predictions = tree.predict(X_test[:,1:5])
predictions_matrix = np.column_stack((identifiers, predictions))


In [28]:
error1 = sklearn.metrics.mean_squared_error(y_test[:,1], predictions)
print (np.sqrt(error1))
print (np.sqrt(error1)/np.mean(predictions))

17.131146793959548
0.2953645998958543


Now that we've done that basic part, let's try some other approaches. First, let's sort by distance. To do this, we'll use sklearn's euclidean distance portion. For the first tree in our dataset:

Next steps: organize the data (150). 

Keep some regularization.

Change the model structure so that it includes physical processes that we know.

Distance as pseudo-time
